## The Keras Functional API

### Input layers

In [3]:
# Import Input from keras.layers
from keras.layers import Input

# Create an input layer of shape 1
input_tensor = Input(shape=(1,))

Using TensorFlow backend.


### Dense layers

In [4]:
# Load layers
from keras.layers import Input, Dense

# Input layer
input_tensor = Input(shape=(1,))

# Dense layer
output_layer = Dense(1)

# Connect the dense layer to the input_tensor
output_tensor = output_layer(input_tensor)

### Output layers

In [5]:
# Load layers
from keras.layers import Input, Dense
# Input layer
input_tensor = Input(shape=(1,))

# Create a dense layer and connect the dense layer to the input_tensor in one step
# Note that we did this in 2 steps in the previous exercise, but are doing it in one step now
output_tensor = Dense(1)(input_tensor)

### Build a model

In [6]:
# Input/dense/output layers
from keras.layers import Input, Dense
input_tensor = Input(shape=(1,))
output_tensor = Dense(1)(input_tensor)

# Build the model
from keras.models import Model
model = Model(input_tensor, output_tensor)

### Compile a model

In [7]:
# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

### Visualize a model

In [21]:
# Import the plotting function
from keras.utils import plot_model
import matplotlib.pyplot as plt

# Summarize the model
model.summary()

# # Plot the model
# plot_model(model, to_file='model.png')

# # Display the image
# data = plt.imread('model.png')
# plt.imshow(data)
# plt.show()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


### Fit the model to the tournament basketball data

In [41]:
import pandas as pd
games_tourney = pd.read_csv('basketball_data/games_tourney.csv')
games_tourney_train = games_tourney.iloc[: 3430, :]
games_tourney_test = games_tourney.iloc[3430:, :]
games_tourney.head()

,season,team_1,team_2,home,seed_diff,score_diff,score_1,score_2,won
0,1985,288,73,0,-3,-9,41,50,0
1,1985,5929,73,0,4,6,61,55,1
2,1985,9884,73,0,5,-4,59,63,0
3,1985,73,288,0,3,9,50,41,1
4,1985,3920,410,0,1,-9,54,63,0


In [18]:
# Now fit the model
model.fit(games_tourney_train['seed_diff'], games_tourney_train['score_diff'],
          epochs=1,
          batch_size=128,
          validation_split=0.1,
          verbose=True)

Train on 3087 samples, validate on 343 samples
Epoch 1/1
3087/3087 [==============================] - 0s 17us/step - loss: 14.1792 - val_loss: 13.2504


### Evaluate the model on a test set

In [19]:
# Load the X variable from the test data
X_test = games_tourney_test['seed_diff']

# Load the y variable from the test data
y_test =games_tourney_test['score_diff']

# Evaluate the model on the test data
model.evaluate(X_test, y_test)

804/804 [==============================] - 0s 21us/step


13.304261093709007

## Two Input Networks Using Categorical Embeddings, Shared Layers, and Merge Layers

### Define team lookup

In [24]:
games_season = pd.read_csv('basketball_data/games_season.csv')
games_season.head()

,season,team_1,team_2,home,score_diff,score_1,score_2,won
0,1985,3745,6664,0,17,81,64,1
1,1985,126,7493,1,7,77,70,1
2,1985,288,3593,1,7,63,56,1
3,1985,1846,9881,1,16,70,54,1
4,1985,2675,10298,1,12,86,74,1


In [25]:
# Imports
from keras.layers import Embedding
from numpy import unique

# Count the unique number of teams
n_teams = unique(games_season['team_1']).shape[0]

# Create an embedding layer
team_lookup = Embedding(input_dim=n_teams,
                        output_dim=1,
                        input_length=1,
                        name='Team-Strength')

### Define team model

In [26]:
# Imports
from keras.layers import Input, Embedding, Flatten
from keras.models import Model

# Create an input layer for the team ID
teamid_in = Input(shape=(1,))

# Lookup the input in the team strength embedding layer
strength_lookup = team_lookup(teamid_in)

# Flatten the output
strength_lookup_flat = Flatten()(strength_lookup)

# Combine the operations into a single, re-usable model
team_strength_model = Model(teamid_in, strength_lookup_flat, name='Team-Strength-Model')

### Defining two inputs

In [29]:
# Load the input layer from keras.layers
from keras.layers import Input

# Input layer for team 1
team_in_1 = Input(shape = (1, ), name = "Team-1-In" )

# Separate input layer for team 2
team_in_2 = Input(shape = (1, ), name = "Team-2-In")

### Lookup both inputs in the same model

In [32]:
# Lookup team 1 in the team strength model
team_1_strength = team_strength_model(team_in_1)

# Lookup team 2 in the team strength model
team_2_strength = team_strength_model(team_in_2)

### Output layer using shared layer

In [33]:
# Import the Subtract layer from keras
from keras.layers import Subtract

# Create a subtract layer using the inputs from the previous exercise
score_diff = Subtract()([team_1_strength,team_2_strength])

### Model using two inputs and one output

In [34]:
# Imports
from keras.layers import Subtract
from keras.models import Model

# Subtraction layer from previous exercise
score_diff = Subtract()([team_1_strength, team_2_strength])

# Create the model
model = Model([team_in_1, team_in_2], score_diff)

# Compile the model
model.compile(optimizer = 'adam', loss = 'mean_absolute_error')

### Fit the model to the regular season training data

In [36]:
# Get the team_1 column from the regular season data
input_1 = games_season['team_1']

# Get the team_2 column from the regular season data
input_2 = games_season['team_2']

# Fit the model to input 1 and 2, using score diff as a target
model.fit([input_1, input_2],
          games_season['score_diff'],
          epochs=1,
          batch_size=2048,
          validation_split=0.10,
          verbose=True)

Train on 280960 samples, validate on 31218 samples
Epoch 1/1
280960/280960 [==============================] - 0s 1us/step - loss: 12.1205 - val_loss: 11.8379


In [38]:
# Get team_1 from the tournament data
input_1 = games_tourney['team_1']

# Get team_2 from the tournament data
input_2 = games_tourney['team_2']

# Evaluate the model using these inputs
model.evaluate([input_1, input_2], games_tourney['score_diff'])

4234/4234 [==============================] - 0s 22us/step


11.681247380526493

## Multiple Inputs: 3 Inputs (and Beyond!)

### Make an input layer for home vs. away

In [40]:
from keras.layers import Concatenate
# Create an Input for each team
team_in_1 = Input(shape=(1,), name='Team-1-In')
team_in_2 = Input(shape=(1,), name='Team-2-In')

# Create an input for home vs away
home_in = Input(shape=(1,), name='Home-In')

# Lookup the team inputs in the team strength model
team_1_strength = team_strength_model(team_in_1)
team_2_strength = team_strength_model(team_in_2)

# Combine the team strengths with the home input using a Concatenate layer, then add a Dense layer
out = Concatenate()([team_1_strength, team_2_strength, home_in])
out = Dense(1)(out)

### Make a model and compile it

In [42]:
# Import the model class
from keras.models import Model

# Make a Model
model = Model([team_in_1, team_in_2, home_in], out)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

### Fit the model and evaluate

In [43]:
# Fit the model to the games_season dataset
model.fit([games_season['team_1'], games_season['team_2'], games_season['home']], games_season['score_diff'], 
          epochs=1,
          verbose=True,
          validation_split=0.1,
          batch_size=2048)

# Evaluate the model on the games_tourney dataset
model.evaluate([games_tourney['team_1'], games_tourney['team_2'], games_tourney['home']], games_tourney['score_diff'])

Train on 280960 samples, validate on 31218 samples
Epoch 1/1
4234/4234 [==============================] - 0s 21us/step


11.689384638840824

### Plotting models

In [ ]:
# Imports
import matplotlib.pyplot as plt
from keras.utils import plot_model

# Plot the model
plot_model(model, to_file='model.png')

# Display the image
data = plt.imread('model.png')
plt.imshow(data)
plt.show()

### Add the model predictions to the tournament data

In [45]:
# Predict
games_tourney['pred'] = model.predict([games_tourney['team_1'], games_tourney['team_2'], games_tourney['home']])

### Create an input layer with multiple columns

In [46]:
 # Create an input layer with 3 columns
input_tensor = Input(shape = (3, ))

# Pass it to a Dense layer with 1 unit
output_tensor = Dense(1)(input_tensor)

# Create a model
model = Model(input_tensor, output_tensor)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

### Fit the model

In [48]:
# Fit the model
games_tourney_train = games_tourney.iloc[: 3430, :]
games_tourney_test = games_tourney.iloc[3430:, :]
model.fit(games_tourney_train[['home', 'seed_diff', 'pred']],
          games_tourney_train['score_diff'],
          epochs=1,
          verbose=True)

Epoch 1/1
3430/3430 [==============================] - 0s 64us/step - loss: 10.2368


### Evaluate the model

In [50]:
# Evaluate the model on the games_tourney_test dataset
model.evaluate(games_tourney_test[['home', 'seed_diff', 'pred']], 
               games_tourney_test['score_diff'])

804/804 [==============================] - 0s 82us/step


9.800892986468416

##  Multiple Outputs


### Simple two-output model

In [51]:
# Define the input
input_tensor = Input(shape = (2, ))

# Define the output
output_tensor = Dense(2)(input_tensor)

# Create a model
model = Model(input_tensor, output_tensor)

# Compile the model
model.compile(optimizer = 'adam', loss = 'mean_absolute_error')

### Fit a model with two outputs

In [52]:
# Fit the model
model.fit(games_tourney_train[['seed_diff','pred']],
  		  games_tourney_train[['score_1', 'score_2']],
  		  verbose=True,
  		  epochs = 100,
  		  batch_size = 16384)

Epoch 1/100
3430/3430 [==============================] - 0s 46us/step - loss: 71.5107
Epoch 2/100
3430/3430 [==============================] - 0s 1us/step - loss: 71.5095
Epoch 3/100
3430/3430 [==============================] - 0s 1us/step - loss: 71.5084
Epoch 4/100
3430/3430 [==============================] - 0s 2us/step - loss: 71.5073
Epoch 5/100
3430/3430 [==============================] - 0s 1us/step - loss: 71.5061
Epoch 6/100
3430/3430 [==============================] - 0s 1us/step - loss: 71.5050
Epoch 7/100
3430/3430 [==============================] - 0s 2us/step - loss: 71.5039
Epoch 8/100
3430/3430 [==============================] - 0s 2us/step - loss: 71.5028
Epoch 9/100
3430/3430 [==============================] - 0s 1us/step - loss: 71.5016
Epoch 10/100
3430/3430 [==============================] - 0s 1us/step - loss: 71.5005
Epoch 11/100
3430/3430 [==============================] - 0s 1us/step - loss: 71.4994
Epoch 12/100
3430/3430 [==============================] - 0s 1

Epoch 97/100
3430/3430 [==============================] - 0s 1us/step - loss: 71.4022
Epoch 98/100
3430/3430 [==============================] - 0s 1us/step - loss: 71.4011
Epoch 99/100
3430/3430 [==============================] - 0s 1us/step - loss: 71.3999
Epoch 100/100
3430/3430 [==============================] - 0s 1us/step - loss: 71.3988


### Inspect the model (I)

In [53]:
# Print the model's weights
print(model.get_weights())

# Print the column means of the training data
print(games_tourney_train.mean())

[array([[-1.2213349 ,  1.1220242 ],
       [ 0.76635975,  1.1487168 ]], dtype=float32), array([0.10007598, 0.10007598], dtype=float32)]
season        1998.074052
team_1        5556.771429
team_2        5556.771429
home             0.000000
seed_diff        0.000000
score_diff       0.000000
score_1         71.621283
score_2         71.621283
won              0.500000
pred             0.129023
dtype: float64


### Evaluate the model

In [54]:
# Evaluate the model on the tournament test data
model.evaluate(games_tourney_test[['seed_diff','pred']], games_tourney_test[['score_1', 'score_2']])


804/804 [==============================] - 0s 67us/step


68.81684845122533

### Classification and regression in one model

In [55]:
# Create an input layer with 2 columns
input_tensor = Input(shape=(2, ))

# Create the first output
output_tensor_1 = Dense(1, activation='linear', use_bias=False)(input_tensor)

# Create the second output (use the first output as input here)
output_tensor_2 = Dense(1, activation='sigmoid', use_bias=False)(output_tensor_1)

# Create a model with 2 outputs
model = Model(input_tensor, [output_tensor_1, output_tensor_2])

### Compile and fit the model

In [56]:
# Import the Adam optimizer
from keras.optimizers import Adam

# Compile the model with 2 losses and the Adam optimzer with a higher learning rate
model.compile(loss=['mean_absolute_error', 'binary_crossentropy'], optimizer=Adam(0.01))

# Fit the model to the tournament training data, with 2 inputs and 2 outputs
model.fit(games_tourney_train[['seed_diff', 'pred']],
          [games_tourney_train[['score_diff']], games_tourney_train[['won']]],
          epochs=10,
          verbose=True,
          batch_size=16384)

Epoch 1/10
3430/3430 [==============================] - 0s 45us/step - loss: 16.7325 - dense_7_loss: 9.2204 - dense_8_loss: 7.5121
Epoch 2/10
3430/3430 [==============================] - 0s 1us/step - loss: 16.6686 - dense_7_loss: 9.2315 - dense_8_loss: 7.4371
Epoch 3/10
3430/3430 [==============================] - 0s 2us/step - loss: 16.6188 - dense_7_loss: 9.2426 - dense_8_loss: 7.3762
Epoch 4/10
3430/3430 [==============================] - 0s 2us/step - loss: 16.5606 - dense_7_loss: 9.2537 - dense_8_loss: 7.3069
Epoch 5/10
3430/3430 [==============================] - 0s 2us/step - loss: 16.5044 - dense_7_loss: 9.2648 - dense_8_loss: 7.2396
Epoch 6/10
3430/3430 [==============================] - 0s 2us/step - loss: 16.4456 - dense_7_loss: 9.2759 - dense_8_loss: 7.1697
Epoch 7/10
3430/3430 [==============================] - 0s 3us/step - loss: 16.3928 - dense_7_loss: 9.2874 - dense_8_loss: 7.1054
Epoch 8/10
3430/3430 [==============================] - 0s 2us/step - loss: 16.3369 - den

### Inspect the model

In [57]:
# Print the model weights
print(model.get_weights())

# Print the training data means
print(games_tourney_train.mean())

[array([[0.88577145],
       [0.13118929]], dtype=float32), array([[-1.6194929]], dtype=float32)]
season        1998.074052
team_1        5556.771429
team_2        5556.771429
home             0.000000
seed_diff        0.000000
score_diff       0.000000
score_1         71.621283
score_2         71.621283
won              0.500000
pred             0.129023
dtype: float64


In [58]:
# Import the sigmoid function from scipy
from scipy.special import expit as sigmoid

# Weight from the model
weight = 0.14

# Print the approximate win probability of a predicted close game
print(sigmoid(1 * weight))

# Print the approximate win probability of a predicted blowout game
print(sigmoid(10 * weight))

0.5349429451582145
0.8021838885585818


### Evaluate on new data with two metrics

In [59]:
# Evaluate the model on new data
model.evaluate(games_tourney_test[['seed_diff', 'pred']],
          [games_tourney_test[['score_diff']], games_tourney_test[['won']]])
               

804/804 [==============================] - 0s 109us/step


[15.595239496942776, 9.223325501626997, 6.371913922959893]